# [Pytorch] Birdclef starter

In [1]:
!pip install timm
!pip install --upgrade wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.5/431.5 KB 4.6 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 6.6 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.3/145.3 KB 2.5 MB/s eta 0:00:000:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 KB 1.5 MB/s eta 0:00:005 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.2 MB/s eta 0:00:00m eta 0:00:010:00:01
  Preparing metadata (setup.py) ... done
  Using cached GitPython-3.1.27-py3-none-any.whl (181 kB)
  Preparing metadata (setup.py) ... done
  Using cached gitdb-4.0.9-py3-none-any.whl (63 kB)
  Using cached smmap-5.0.0-py3-none-any.whl (24 kB)
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21503 sha256=f8c595b033bce360aefedc2c09d434c220be03913b7724138dc15233f89b6806
  Stored in directory: /home/skang/.cache/pip/wheels/54/aa/01/724885182f93150035a2a91bce34a12877e8067a97baaf5dc8
  Created 

In [6]:
import os
import gc
import cv2
import math
import copy
import time
import random

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp

# Audio 
import torchaudio
from torchaudio.transforms import MelSpectrogram, Resample

# Utils
import joblib
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

# For Image Models
import timm

# For colored terminal text
from colorama import Fore, Back, Style
b_ = Fore.BLUE
sr_ = Style.RESET_ALL

import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

**import wandb and log in**     
You will need a unique API key to log in to Weights & Biases.

If you don't have a Weights & Biases account, you can go to https://wandb.ai/site and create a FREE account.
Access your API key: https://wandb.ai/authorize.

In [21]:
import wandb

wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [15]:
CONFIG = {"seed": 2022,
          "epochs": 10,
          "model_name": "tf_efficientnet_b0_ns",
          "embedding_size": 768,
          "num_classes": 152,
          "train_batch_size": 64,
          "valid_batch_size": 128,
          "learning_rate": 1e-4,
          "scheduler": 'CosineAnnealingLR',
          "min_lr": 1e-6,
          "T_max": 500,
          "weight_decay": 1e-6,
          "n_fold": 5,
          "n_accumulate": 1,
          "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
          "competition": "BirdCLEF22",
          "_wandb_kernel": "deb",
          # Audio Specific
          "sample_rate": 32_000,
          "max_time": 5,
          "n_mels": 224,
          "n_fft": 1024,
          }

In [16]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(CONFIG['seed'])

In [17]:
ROOT_DIR = '../input/birdclef-2022'
TRAIN_DIR = '../input/birdclef-2022/train_audio'
TEST_DIR = '../input/birdclef-2022/test_soundscapes'

In [18]:
def get_train_file_path(filename):
    return f"{TRAIN_DIR}/{filename}"

# Read data

In [19]:
df = pd.read_csv(f"{ROOT_DIR}/train_metadata.csv")
df['file_path'] = df['filename'].apply(get_train_file_path)
df.head()

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,time,url,filename,file_path
0,afrsil1,[],"['call', 'flight call']",12.3910,-1.4930,Euodice cantans,African Silverbill,Bram Piot,Creative Commons Attribution-NonCommercial-Sha...,2.5,08:00,https://www.xeno-canto.org/125458,afrsil1/XC125458.ogg,../input/birdclef-2022/train_audio/afrsil1/XC1...
1,afrsil1,"['houspa', 'redava', 'zebdov']",['call'],19.8801,-155.7254,Euodice cantans,African Silverbill,Dan Lane,Creative Commons Attribution-NonCommercial-Sha...,3.5,08:30,https://www.xeno-canto.org/175522,afrsil1/XC175522.ogg,../input/birdclef-2022/train_audio/afrsil1/XC1...
2,afrsil1,[],"['call', 'song']",16.2901,-16.0321,Euodice cantans,African Silverbill,Bram Piot,Creative Commons Attribution-NonCommercial-Sha...,4.0,11:30,https://www.xeno-canto.org/177993,afrsil1/XC177993.ogg,../input/birdclef-2022/train_audio/afrsil1/XC1...
3,afrsil1,[],"['alarm call', 'call']",17.0922,54.2958,Euodice cantans,African Silverbill,Oscar Campbell,Creative Commons Attribution-NonCommercial-Sha...,4.0,11:00,https://www.xeno-canto.org/205893,afrsil1/XC205893.ogg,../input/birdclef-2022/train_audio/afrsil1/XC2...
4,afrsil1,[],['flight call'],21.4581,-157.7252,Euodice cantans,African Silverbill,Ross Gallardy,Creative Commons Attribution-NonCommercial-Sha...,3.0,16:30,https://www.xeno-canto.org/207431,afrsil1/XC207431.ogg,../input/birdclef-2022/train_audio/afrsil1/XC2...


## Visualize images

In [25]:
run = wandb.init(project=CONFIG['competition'],
                 job_type='Visualization',
                 name='Audio Visualization',
                 )

In [26]:
preview_table = wandb.Table(columns=['Audio', 'Label', 'Rating', 'Time'])

temp_df = df.sample(5000).reset_index(drop=True)

for i in tqdm(range(len(temp_df))):
    row = temp_df.loc[i]
    audio = wandb.Audio(row.file_path, sample_rate=CONFIG['sample_rate'])
    preview_table.add_data(audio,
                           row.primary_label,
                           row.rating,
                           row.time)

wandb.log({'Visualization': preview_table})
run.finish()

100%|█| 5000/5000


인코더는 사용할 때마다 다르게 인코딩을 할 수 있으므로 재현성을 위하여 저장해둔다.

In [28]:
encoder = LabelEncoder()
df['primary_label'] = encoder.fit_transform(df['primary_label'])

with open('le.pkl', "wb") as fp:
    joblib.dump(encoder, fp)

# Create folds

각 새 종류가 train, val에 적절한 비율로 들어가도록 하기 위해 `StratifiedKFold`를 이용한다.

In [29]:
skf = StratifiedKFold(n_splits=CONFIG['n_fold'])

for fold, ( _, val_) in enumerate(skf.split(X=df, y=df.primary_label)):
      df.loc[val_ , "kfold"] = fold

# Dataset Class

In [30]:
df['file_path']

0        ../input/birdclef-2022/train_audio/afrsil1/XC1...
1        ../input/birdclef-2022/train_audio/afrsil1/XC1...
2        ../input/birdclef-2022/train_audio/afrsil1/XC1...
3        ../input/birdclef-2022/train_audio/afrsil1/XC2...
4        ../input/birdclef-2022/train_audio/afrsil1/XC2...
                               ...                        
14847    ../input/birdclef-2022/train_audio/zebdov/XC62...
14848    ../input/birdclef-2022/train_audio/zebdov/XC64...
14849    ../input/birdclef-2022/train_audio/zebdov/XC66...
14850    ../input/birdclef-2022/train_audio/zebdov/XC66...
14851    ../input/birdclef-2022/train_audio/zebdov/XC66...
Name: file_path, Length: 14852, dtype: object

In [54]:
class BirdCLEFDataset(Dataset):
    def __init__(self, df, target_sample_rate, max_time, image_transforms=None):
        self.file_paths = df['file_path'].values # 오디오 파일 경로
        self.labels = df['primary_label'].values # 오디오 파일의 label(target)
        self.target_sample_rate = target_sample_rate # 모델에 input으로 들어갈 파일의 sampling rate (오디오 파일마다 다를 수 있으므로 모델에 들어가기 전 맞춰준다.)
        num_samples = target_sample_rate * max_time # 샘플의 전체 데이터 포인트 개수 (sr (data / time) * time(time))
        self.num_samples = num_samples 
        self.image_transforms = image_transforms 
        
        
    def __len__(self):
        return len(self.file_paths)
    
    
    def __getitem__(self, index):
        filepath = self.file_paths[index] 
        audio, sample_rate = torchaudio.load(filepath) # torchaudio의 반환 결과 (sample, sample_rate)
        audio = self.to_mono(audio) # 채널 수가 여러 개일 수 있으므로 평균을 통해 하나로 맞춰준다.
        
        if sample_rate != self.target_sample_rate:
            resample = Resample(sample_rate, self.target_sample_rate) # 원본 데이터의 sampling rate가 모델의 것과 맞지 않을 경우 바꾸어준다.
            audio = resample(audio)
        
        if audio.shape[0] > self.num_samples: # 오디오의 샘플 수가 max보다 클 경우 자른다. 
            audio = self.crop_audio(audio)
            
        if audio.shape[0] < self.num_samples: # 오디오가 짧을 경우 padding을 넣어준다.
            audio = self.pad_audio(audio)
            
        mel_spectogram = MelSpectrogram(sample_rate=self.target_sample_rate, # mel스펙트로그램으로 변환
                                        n_mels=CONFIG['n_mels'], 
                                        n_fft=CONFIG['n_fft'])
        mel = mel_spectogram(audio)
        label = torch.tensor(self.labels[index])
        
        # Convert to Image
        image = torch.stack([mel, mel, mel]) # 사용하려는 사전 훈련 모델의 input차원이 3차원이므로 차원수를 3개로 늘린다.
        
        # Normalize Image
        max_val = torch.abs(image).max() # 각 샘플의 max값을 나눠줌으로써 normalize
        image = image / max_val 
        
        return {
            "image": image, 
            "label": label
        }
    
    
    def pad_audio(self, audio):
        """_summary_
        길이가 짧은 오디오 샘플을 패딩(뒤쪽에 필요한 만큼 0을 추가한다.)

        Args:
            audio (torch.Tensor): 오디오 샘플 (1D, [length_of_samples])

        Returns:
            torch.Tensor: 패딩된 오디오 샘플 (1D, [self.num_samples])
        """
        pad_length = self.num_samples - audio.shape[0]
        last_dim_padding = (0, pad_length) # (pad_left, pad_right)
        audio = F.pad(audio, last_dim_padding)
        return audio
    
    
    def crop_audio(self, audio):
        """_summary_
        길이가 긴 audio sample을 자른다.(뒷부분을 자른다.)
        Args:
            audio (torch.Tensor): 오디오 샘플 (1D, [length_of_samples])

        Returns:
            torch.Tensor: 잘린 오디오 샘플 (1D, [self.num_samples])
        """
        return audio[:self.num_samples]
        
    
    def to_mono(self, audio):
        return torch.mean(audio, axis=0)

# GeM Pooling

In [ ]:
class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM, self).__init__()
        self.p = nn.Parameter(torch.ones(1)*p)
        self.eps = eps

    def forward(self, x):
        return self.gem(x, p=self.p, eps=self.eps)
        
    def gem(self, x, p=3, eps=1e-6):
        return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)
        
    def __repr__(self):
        return self.__class__.__name__ + \
                '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + \
                ', ' + 'eps=' + str(self.eps) + ')'